In [ ]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from matplotlib.colors import LinearSegmentedColormap

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

#create func to compute PR
def comp_PR(ctl,scen,q):
    #first get quantile from ctl
    qctl=ctl.quantile(q,dim='time')
    
    del ctl
    gc.collect()
    

    #get subsample scenarios
    qscen=xr.where(scen>qctl,1,0)
    pscen=qscen.sum('time')/len(qscen.time)
    
    del qscen
    gc.collect()

    #compute PR
    PR_scen=pscen/(1-q)
    PR_scen = np.where(data_mask==0, np.nan, PR_scen)
    PR_scen = np.where(PR_scen==0, np.nan, PR_scen)
    
    return PR_scen

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, :]
    var[np.isnan(var)] = 1
    return var

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]
data_time = np.array(range(262800))

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

def get_data_for_this_study(metric):
    
    PR1_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_all_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
    PR1_CNRM= np.array(PR1_CNRM)
    PR1_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_NorESM= np.array(PR1_NorESM)
    PR1_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_CESM2= np.array(PR1_CESM2)
    PR1_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_CESM2_gw= np.array(PR1_CESM2_gw)
    PR1_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
    PR1_IPSL= np.array(PR1_IPSL)
#     PR1_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
#     PR1_MIROC= np.array(PR1_MIROC)
    PR1_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
    PR1_E3SM= np.array(PR1_E3SM)
    
    PR1_consis_pos = np.zeros([163, 288])
    PR1_consis_neg = np.zeros([163, 288])
    PR1_consis_pos[PR1_CNRM>=1.5] = PR1_consis_pos[PR1_CNRM>=1.5]+1
    PR1_consis_neg[PR1_CNRM<2/3] = PR1_consis_neg[PR1_CNRM<2/3]+1
    
    PR1_consis_pos[PR1_NorESM>=1.5] = PR1_consis_pos[PR1_NorESM>=1.5]+1
    PR1_consis_neg[PR1_NorESM<2/3] = PR1_consis_neg[PR1_NorESM<2/3]+1
    
    PR1_consis_pos[PR1_CESM2>=1.5] = PR1_consis_pos[PR1_CESM2>=1.5]+1
    PR1_consis_neg[PR1_CESM2<2/3] = PR1_consis_neg[PR1_CESM2<2/3]+1
    
    PR1_consis_pos[PR1_CESM2_gw>=1.5] = PR1_consis_pos[PR1_CESM2_gw>=1.5]+1
    PR1_consis_neg[PR1_CESM2_gw<2/3] = PR1_consis_neg[PR1_CESM2_gw<2/3]+1
    
    PR1_consis_pos[PR1_IPSL>=1.5] = PR1_consis_pos[PR1_IPSL>=1.5]+1
    PR1_consis_neg[PR1_IPSL<2/3] = PR1_consis_neg[PR1_IPSL<2/3]+1
    
#     PR1_consis_pos[PR1_MIROC>=1.5] = PR1_consis_pos[PR1_MIROC>=1.5]+1
#     PR1_consis_neg[PR1_MIROC<2/3] = PR1_consis_neg[PR1_MIROC<2/3]+1
    
    PR1_consis_pos[PR1_E3SM>=1.5] = PR1_consis_pos[PR1_E3SM>=1.5]+1
    PR1_consis_neg[PR1_E3SM<2/3] = PR1_consis_neg[PR1_E3SM<2/3]+1
    
    PR1_consis = np.maximum(PR1_consis_pos, PR1_consis_neg)
    
    scio.savemat("/dodrio/scratch/users/vsc10154/PR1_consis.mat", {'PR1_consis': PR1_consis})
    
    PR1 = np.power(PR1_CNRM*PR1_NorESM*PR1_CESM2*PR1_CESM2_gw*PR1_IPSL*PR1_E3SM, 1/6)
#     PR1 = np.power(PR1_NorESM*PR1_CESM2*PR1_CESM2_gw*PR1_IPSL*PR1_MIROC, 1/5)
    PR1[data_mask==0] = np.nan
    
    
    
    
    
    PR2_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_oth_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
    PR2_CNRM= np.array(PR2_CNRM)
    PR2_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_NorESM= np.array(PR2_NorESM)
    PR2_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_CESM2= np.array(PR2_CESM2)
    PR2_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_CESM2_gw= np.array(PR2_CESM2_gw)
    PR2_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
    PR2_IPSL= np.array(PR2_IPSL)
#     PR2_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
#     PR2_MIROC= np.array(PR2_MIROC)
    PR2_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
    PR2_E3SM= np.array(PR2_E3SM)
    PR2 = np.power(PR2_CNRM*PR2_NorESM*PR2_CESM2*PR2_CESM2_gw*PR2_IPSL*PR2_E3SM, 1/6)
#     PR2 = np.power(PR2_NorESM*PR2_CESM2*PR2_CESM2_gw*PR2_IPSL*PR2_MIROC, 1/5)
    
    PR2[data_mask==0] = np.nan
    
    PR2_consis_pos = np.zeros([163, 288])
    PR2_consis_neg = np.zeros([163, 288])
    PR2_consis_pos[PR2_CNRM>=1.5] = PR2_consis_pos[PR2_CNRM>=1.5]+1
    PR2_consis_neg[PR2_CNRM<2/3] = PR2_consis_neg[PR2_CNRM<2/3]+1
    
    PR2_consis_pos[PR2_NorESM>=1.5] = PR2_consis_pos[PR2_NorESM>=1.5]+1
    PR2_consis_neg[PR2_NorESM<2/3] = PR2_consis_neg[PR2_NorESM<2/3]+1
    
    PR2_consis_pos[PR2_CESM2>=1.5] = PR2_consis_pos[PR2_CESM2>=1.5]+1
    PR2_consis_neg[PR2_CESM2<2/3] = PR2_consis_neg[PR2_CESM2<2/3]+1
    
    PR2_consis_pos[PR2_CESM2_gw>=1.5] = PR2_consis_pos[PR2_CESM2_gw>=1.5]+1
    PR2_consis_neg[PR2_CESM2_gw<2/3] = PR2_consis_neg[PR2_CESM2_gw<2/3]+1
    
    PR2_consis_pos[PR2_IPSL>=1.5] = PR2_consis_pos[PR2_IPSL>=1.5]+1
    PR2_consis_neg[PR2_IPSL<2/3] = PR2_consis_neg[PR2_IPSL<2/3]+1
    
#     PR2_consis_pos[PR2_MIROC>=1.5] = PR2_consis_pos[PR2_MIROC>=1.5]+1
#     PR2_consis_neg[PR2_MIROC<2/3] = PR2_consis_neg[PR2_MIROC<2/3]+1
    
    PR2_consis_pos[PR2_E3SM>=1.5] = PR2_consis_pos[PR2_E3SM>=1.5]+1
    PR2_consis_neg[PR2_E3SM<2/3] = PR2_consis_neg[PR2_E3SM<2/3]+1
    
    PR2_consis = np.maximum(PR2_consis_pos, PR2_consis_neg)
    
    
    PR3_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_irr_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
    PR3_CNRM= np.array(PR3_CNRM)
    PR3_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_NorESM= np.array(PR3_NorESM)
    PR3_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_CESM2= np.array(PR3_CESM2)
    PR3_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_CESM2_gw= np.array(PR3_CESM2_gw)
    PR3_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
    PR3_IPSL= np.array(PR3_IPSL)
#     PR3_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
#     PR3_MIROC= np.array(PR3_MIROC)
    PR3_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
    PR3_E3SM= np.array(PR3_E3SM)
    PR3 = np.power(PR3_CNRM*PR3_NorESM*PR3_CESM2*PR3_CESM2_gw*PR3_IPSL*PR3_E3SM, 1/6)
#     PR3 = np.power(PR3_NorESM*PR3_CESM2*PR3_CESM2_gw*PR3_IPSL*PR3_MIROC, 1/5)
    
    PR3[data_mask==0] = np.nan
    
    PR3_consis_pos = np.zeros([163, 288])
    PR3_consis_neg = np.zeros([163, 288])
    PR3_consis_pos[PR3_CNRM>=1.5] = PR3_consis_pos[PR3_CNRM>=1.5]+1
    PR3_consis_neg[PR3_CNRM<2/3] = PR3_consis_neg[PR3_CNRM<2/3]+1
    
    PR3_consis_pos[PR3_NorESM>=1.5] = PR3_consis_pos[PR3_NorESM>=1.5]+1
    PR3_consis_neg[PR3_NorESM<2/3] = PR3_consis_neg[PR3_NorESM<2/3]+1
    
    PR3_consis_pos[PR3_CESM2>=1.5] = PR3_consis_pos[PR3_CESM2>=1.5]+1
    PR3_consis_neg[PR3_CESM2<2/3] = PR3_consis_neg[PR3_CESM2<2/3]+1
    
    PR3_consis_pos[PR3_CESM2_gw>=1.5] = PR3_consis_pos[PR3_CESM2_gw>=1.5]+1
    PR3_consis_neg[PR3_CESM2_gw<2/3] = PR3_consis_neg[PR3_CESM2_gw<2/3]+1
    
    PR3_consis_pos[PR3_IPSL>=1.5] = PR3_consis_pos[PR3_IPSL>=1.5]+1
    PR3_consis_neg[PR3_IPSL<2/3] = PR3_consis_neg[PR3_IPSL<2/3]+1
    
#     PR3_consis_pos[PR3_MIROC>=1.5] = PR3_consis_pos[PR3_MIROC>=1.5]+1
#     PR3_consis_neg[PR3_MIROC<2/3] = PR3_consis_neg[PR3_MIROC<2/3]+1
    
    PR3_consis_pos[PR3_E3SM>=1.5] = PR3_consis_pos[PR3_E3SM>=1.5]+1
    PR3_consis_neg[PR3_E3SM<2/3] = PR3_consis_neg[PR3_E3SM<2/3]+1
    
    PR3_consis = np.maximum(PR3_consis_pos, PR3_consis_neg)
    
    
    PR4_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_all_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
    PR4_CNRM= np.array(PR4_CNRM)
    PR4_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_NorESM= np.array(PR4_NorESM)
    PR4_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_CESM2= np.array(PR4_CESM2)
    PR4_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_CESM2_gw= np.array(PR4_CESM2_gw)
    PR4_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
    PR4_IPSL= np.array(PR4_IPSL)
#     PR4_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
#     PR4_MIROC= np.array(PR4_MIROC)
    PR4_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
    PR4_E3SM= np.array(PR4_E3SM)
    PR4 = np.power(PR4_CNRM*PR4_NorESM*PR4_CESM2*PR4_CESM2_gw*PR4_IPSL*PR4_E3SM, 1/6)
#     PR4 = np.power(PR4_NorESM*PR4_CESM2*PR4_CESM2_gw*PR4_IPSL*PR4_MIROC, 1/5)
    PR4[data_mask==0] = np.nan
    
    PR4_consis_pos = np.zeros([163, 288])
    PR4_consis_neg = np.zeros([163, 288])
    PR4_consis_pos[PR4_CNRM>=1.5] = PR4_consis_pos[PR4_CNRM>=1.5]+1
    PR4_consis_neg[PR4_CNRM<2/3] = PR4_consis_neg[PR4_CNRM<2/3]+1
    
    PR4_consis_pos[PR4_NorESM>=1.5] = PR4_consis_pos[PR4_NorESM>=1.5]+1
    PR4_consis_neg[PR4_NorESM<2/3] = PR4_consis_neg[PR4_NorESM<2/3]+1
    
    PR4_consis_pos[PR4_CESM2>=1.5] = PR4_consis_pos[PR4_CESM2>=1.5]+1
    PR4_consis_neg[PR4_CESM2<2/3] = PR4_consis_neg[PR4_CESM2<2/3]+1
    
    PR4_consis_pos[PR4_CESM2_gw>=1.5] = PR4_consis_pos[PR4_CESM2_gw>=1.5]+1
    PR4_consis_neg[PR4_CESM2_gw<2/3] = PR4_consis_neg[PR4_CESM2_gw<2/3]+1
    
    PR4_consis_pos[PR4_IPSL>=1.5] = PR4_consis_pos[PR4_IPSL>=1.5]+1
    PR4_consis_neg[PR4_IPSL<2/3] = PR4_consis_neg[PR4_IPSL<2/3]+1
    
#     PR4_consis_pos[PR4_MIROC>=1.5] = PR4_consis_pos[PR4_MIROC>=1.5]+1
#     PR4_consis_neg[PR4_MIROC<2/3] = PR4_consis_neg[PR4_MIROC<2/3]+1
    
    PR4_consis_pos[PR4_E3SM>=1.5] = PR4_consis_pos[PR4_E3SM>=1.5]+1
    PR4_consis_neg[PR4_E3SM<2/3] = PR4_consis_neg[PR4_E3SM<2/3]+1
    
    PR4_consis = np.maximum(PR4_consis_pos, PR4_consis_neg)
    
    
    
    PR5_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_oth_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
    PR5_CNRM= np.array(PR5_CNRM)
    PR5_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_NorESM= np.array(PR5_NorESM)
    PR5_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_CESM2= np.array(PR5_CESM2)
    PR5_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_CESM2_gw= np.array(PR5_CESM2_gw)
    PR5_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
    PR5_IPSL= np.array(PR5_IPSL)
#     PR5_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
#     PR5_MIROC= np.array(PR5_MIROC)
    PR5_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
    PR5_E3SM= np.array(PR5_E3SM)
    PR5 = np.power(PR5_CNRM*PR5_NorESM*PR5_CESM2*PR5_CESM2_gw*PR5_IPSL*PR5_E3SM, 1/6)
#     PR5 = np.power(PR5_NorESM*PR5_CESM2*PR5_CESM2_gw*PR5_IPSL*PR5_MIROC, 1/5)
    PR5[data_mask==0] = np.nan
    
    
    PR5_consis_pos = np.zeros([163, 288])
    PR5_consis_neg = np.zeros([163, 288])
    PR5_consis_pos[PR5_CNRM>=1.5] = PR5_consis_pos[PR5_CNRM>=1.5]+1
    PR5_consis_neg[PR5_CNRM<2/3] = PR5_consis_neg[PR5_CNRM<2/3]+1
    
    PR5_consis_pos[PR5_NorESM>=1.5] = PR5_consis_pos[PR5_NorESM>=1.5]+1
    PR5_consis_neg[PR5_NorESM<2/3] = PR5_consis_neg[PR5_NorESM<2/3]+1
    
    PR5_consis_pos[PR5_CESM2>=1.5] = PR5_consis_pos[PR5_CESM2>=1.5]+1
    PR5_consis_neg[PR5_CESM2<2/3] = PR5_consis_neg[PR5_CESM2<2/3]+1
    
    PR5_consis_pos[PR5_CESM2_gw>=1.5] = PR5_consis_pos[PR5_CESM2_gw>=1.5]+1
    PR5_consis_neg[PR5_CESM2_gw<2/3] = PR5_consis_neg[PR5_CESM2_gw<2/3]+1
    
    PR5_consis_pos[PR5_IPSL>=1.5] = PR5_consis_pos[PR5_IPSL>=1.5]+1
    PR5_consis_neg[PR5_IPSL<2/3] = PR5_consis_neg[PR5_IPSL<2/3]+1
    
#     PR5_consis_pos[PR5_MIROC>=1.5] = PR5_consis_pos[PR5_MIROC>=1.5]+1
#     PR5_consis_neg[PR5_MIROC<2/3] = PR5_consis_neg[PR5_MIROC<2/3]+1
    
    PR5_consis_pos[PR5_E3SM>=1.5] = PR5_consis_pos[PR5_E3SM>=1.5]+1
    PR5_consis_neg[PR5_E3SM<2/3] = PR5_consis_neg[PR5_E3SM<2/3]+1
    
    PR5_consis = np.maximum(PR5_consis_pos, PR5_consis_neg)
    
    
    PR6_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_irr_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
    PR6_CNRM= np.array(PR6_CNRM)
    PR6_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_NorESM= np.array(PR6_NorESM)
    PR6_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_CESM2= np.array(PR6_CESM2)
    PR6_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_CESM2_gw= np.array(PR6_CESM2_gw)
    PR6_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
    PR6_IPSL= np.array(PR6_IPSL)
#     PR6_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
#     PR6_MIROC= np.array(PR6_MIROC)
    PR6_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
    PR6_E3SM= np.array(PR6_E3SM)
    PR6 = np.power(PR6_CNRM*PR6_NorESM*PR6_CESM2*PR6_CESM2_gw*PR6_IPSL*PR6_E3SM, 1/6)
#     PR6 = np.power(PR6_NorESM*PR6_CESM2*PR6_CESM2_gw*PR6_IPSL*PR6_MIROC, 1/5)
    PR6[data_mask==0] = np.nan


    PR6_consis_pos = np.zeros([163, 288])
    PR6_consis_neg = np.zeros([163, 288])
    PR6_consis_pos[PR6_CNRM>=1.5] = PR6_consis_pos[PR6_CNRM>=1.5]+1
    PR6_consis_neg[PR6_CNRM<2/3] = PR6_consis_neg[PR6_CNRM<2/3]+1
    
    PR6_consis_pos[PR6_NorESM>=1.5] = PR6_consis_pos[PR6_NorESM>=1.5]+1
    PR6_consis_neg[PR6_NorESM<2/3] = PR6_consis_neg[PR6_NorESM<2/3]+1
    
    PR6_consis_pos[PR6_CESM2>=1.5] = PR6_consis_pos[PR6_CESM2>=1.5]+1
    PR6_consis_neg[PR6_CESM2<2/3] = PR6_consis_neg[PR6_CESM2<2/3]+1
    
    PR6_consis_pos[PR6_CESM2_gw>=1.5] = PR6_consis_pos[PR6_CESM2_gw>=1.5]+1
    PR6_consis_neg[PR6_CESM2_gw<2/3] = PR6_consis_neg[PR6_CESM2_gw<2/3]+1
    
    PR6_consis_pos[PR6_IPSL>=1.5] = PR6_consis_pos[PR6_IPSL>=1.5]+1
    PR6_consis_neg[PR6_IPSL<2/3] = PR6_consis_neg[PR6_IPSL<2/3]+1
    
#     PR6_consis_pos[PR6_MIROC>=1.5] = PR6_consis_pos[PR6_MIROC>=1.5]+1
#     PR6_consis_neg[PR6_MIROC<2/3] = PR6_consis_neg[PR6_MIROC<2/3]+1
    
    PR6_consis_pos[PR6_E3SM>=1.5] = PR6_consis_pos[PR6_E3SM>=1.5]+1
    PR6_consis_neg[PR6_E3SM<2/3] = PR6_consis_neg[PR6_E3SM<2/3]+1
    
    PR6_consis = np.maximum(PR6_consis_pos, PR6_consis_neg)

    
    
    PR7_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_all_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
    PR7_CNRM= np.array(PR7_CNRM)
    PR7_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_NorESM= np.array(PR7_NorESM)
    PR7_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_CESM2= np.array(PR7_CESM2)
    PR7_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_CESM2_gw= np.array(PR7_CESM2_gw)
    PR7_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
    PR7_IPSL= np.array(PR7_IPSL)
#     PR7_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
#     PR7_MIROC= np.array(PR7_MIROC)
    PR7_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
    PR7_E3SM= np.array(PR7_E3SM)
    PR7 = np.power(PR7_CNRM*PR7_NorESM*PR7_CESM2*PR7_CESM2_gw*PR7_IPSL*PR7_E3SM, 1/6)
#     PR7 = np.power(PR7_NorESM*PR7_CESM2*PR7_CESM2_gw*PR7_IPSL*PR7_MIROC, 1/5)
    PR7[data_mask==0] = np.nan
    
    
    PR7_consis_pos = np.zeros([163, 288])
    PR7_consis_neg = np.zeros([163, 288])
    PR7_consis_pos[PR7_CNRM>=1.5] = PR7_consis_pos[PR7_CNRM>=1.5]+1
    PR7_consis_neg[PR7_CNRM<2/3] = PR7_consis_neg[PR7_CNRM<2/3]+1
    
    PR7_consis_pos[PR7_NorESM>=1.5] = PR7_consis_pos[PR7_NorESM>=1.5]+1
    PR7_consis_neg[PR7_NorESM<2/3] = PR7_consis_neg[PR7_NorESM<2/3]+1
    
    PR7_consis_pos[PR7_CESM2>=1.5] = PR7_consis_pos[PR7_CESM2>=1.5]+1
    PR7_consis_neg[PR7_CESM2<2/3] = PR7_consis_neg[PR7_CESM2<2/3]+1
    
    PR7_consis_pos[PR7_CESM2_gw>=1.5] = PR7_consis_pos[PR7_CESM2_gw>=1.5]+1
    PR7_consis_neg[PR7_CESM2_gw<2/3] = PR7_consis_neg[PR7_CESM2_gw<2/3]+1
    
    PR7_consis_pos[PR7_IPSL>=1.5] = PR7_consis_pos[PR7_IPSL>=1.5]+1
    PR7_consis_neg[PR7_IPSL<2/3] = PR7_consis_neg[PR7_IPSL<2/3]+1
    
#     PR7_consis_pos[PR7_MIROC>=1.5] = PR7_consis_pos[PR7_MIROC>=1.5]+1
#     PR7_consis_neg[PR7_MIROC<2/3] = PR7_consis_neg[PR7_MIROC<2/3]+1
    
    PR7_consis_pos[PR7_E3SM>=1.5] = PR7_consis_pos[PR7_E3SM>=1.5]+1
    PR7_consis_neg[PR7_E3SM<2/3] = PR7_consis_neg[PR7_E3SM<2/3]+1
    
    PR7_consis = np.maximum(PR7_consis_pos, PR7_consis_neg)
    
    
    PR8_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_oth_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
    PR8_CNRM= np.array(PR8_CNRM)
    PR8_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_NorESM= np.array(PR8_NorESM)
    PR8_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_CESM2= np.array(PR8_CESM2)
    PR8_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_CESM2_gw= np.array(PR8_CESM2_gw)
    PR8_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
    PR8_IPSL= np.array(PR8_IPSL)
#     PR8_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
#     PR8_MIROC= np.array(PR8_MIROC)
    PR8_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
    PR8_E3SM= np.array(PR8_E3SM)
    PR8 = np.power(PR8_CNRM*PR8_NorESM*PR8_CESM2*PR8_CESM2_gw*PR8_IPSL*PR8_E3SM, 1/6)
#     PR8 = np.power(PR8_NorESM*PR8_CESM2*PR8_CESM2_gw*PR8_IPSL*PR8_MIROC, 1/5)
    PR8[data_mask==0] = np.nan
    
    
    PR8_consis_pos = np.zeros([163, 288])
    PR8_consis_neg = np.zeros([163, 288])
    PR8_consis_pos[PR8_CNRM>=1.5] = PR8_consis_pos[PR8_CNRM>=1.5]+1
    PR8_consis_neg[PR8_CNRM<2/3] = PR8_consis_neg[PR8_CNRM<2/3]+1
    
    PR8_consis_pos[PR8_NorESM>=1.5] = PR8_consis_pos[PR8_NorESM>=1.5]+1
    PR8_consis_neg[PR8_NorESM<2/3] = PR8_consis_neg[PR8_NorESM<2/3]+1
    
    PR8_consis_pos[PR8_CESM2>=1.5] = PR8_consis_pos[PR8_CESM2>=1.5]+1
    PR8_consis_neg[PR8_CESM2<2/3] = PR8_consis_neg[PR8_CESM2<2/3]+1
    
    PR8_consis_pos[PR8_CESM2_gw>=1.5] = PR8_consis_pos[PR8_CESM2_gw>=1.5]+1
    PR8_consis_neg[PR8_CESM2_gw<2/3] = PR8_consis_neg[PR8_CESM2_gw<2/3]+1
    
    PR8_consis_pos[PR8_IPSL>=1.5] = PR8_consis_pos[PR8_IPSL>=1.5]+1
    PR8_consis_neg[PR8_IPSL<2/3] = PR8_consis_neg[PR8_IPSL<2/3]+1
    
#     PR8_consis_pos[PR8_MIROC>=1.5] = PR8_consis_pos[PR8_MIROC>=1.5]+1
#     PR8_consis_neg[PR8_MIROC<2/3] = PR8_consis_neg[PR8_MIROC<2/3]+1
    
    PR8_consis_pos[PR8_E3SM>=1.5] = PR8_consis_pos[PR8_E3SM>=1.5]+1
    PR8_consis_neg[PR8_E3SM<2/3] = PR8_consis_neg[PR8_E3SM<2/3]+1
    
    PR8_consis = np.maximum(PR8_consis_pos, PR8_consis_neg)
    
    
    PR9_CNRM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/CNRM/PR_irr_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
    PR9_CNRM= np.array(PR9_CNRM)
    PR9_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_NorESM= np.array(PR9_NorESM)
    PR9_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_CESM2= np.array(PR9_CESM2)
    PR9_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_CESM2_gw= np.array(PR9_CESM2_gw)
    PR9_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
    PR9_IPSL= np.array(PR9_IPSL)
#     PR9_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
#     PR9_MIROC= np.array(PR9_MIROC)
    PR9_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
    PR9_E3SM= np.array(PR9_E3SM)
    
    PR9_consis_pos = np.zeros([163, 288])
    PR9_consis_neg = np.zeros([163, 288])
    PR9_consis_pos[PR9_CNRM>=1.5] = PR9_consis_pos[PR9_CNRM>=1.5]+1
    PR9_consis_neg[PR9_CNRM<2/3] = PR9_consis_neg[PR9_CNRM<2/3]+1
    
    PR9_consis_pos[PR9_NorESM>=1.5] = PR9_consis_pos[PR9_NorESM>=1.5]+1
    PR9_consis_neg[PR9_NorESM<2/3] = PR9_consis_neg[PR9_NorESM<2/3]+1
    
    PR9_consis_pos[PR9_CESM2>=1.5] = PR9_consis_pos[PR9_CESM2>=1.5]+1
    PR9_consis_neg[PR9_CESM2<2/3] = PR9_consis_neg[PR9_CESM2<2/3]+1
    
    PR9_consis_pos[PR9_CESM2_gw>=1.5] = PR9_consis_pos[PR9_CESM2_gw>=1.5]+1
    PR9_consis_neg[PR9_CESM2_gw<2/3] = PR9_consis_neg[PR9_CESM2_gw<2/3]+1
    
    PR9_consis_pos[PR9_IPSL>=1.5] = PR9_consis_pos[PR9_IPSL>=1.5]+1
    PR9_consis_neg[PR9_IPSL<2/3] = PR9_consis_neg[PR9_IPSL<2/3]+1
    
#     PR9_consis_pos[PR9_MIROC>=1.5] = PR9_consis_pos[PR9_MIROC>=1.5]+1
#     PR9_consis_neg[PR9_MIROC<2/3] = PR9_consis_neg[PR9_MIROC<2/3]+1
    
    PR9_consis_pos[PR9_E3SM>=1.5] = PR9_consis_pos[PR9_E3SM>=1.5]+1
    PR9_consis_neg[PR9_E3SM<2/3] = PR9_consis_neg[PR9_E3SM<2/3]+1
    
    PR9_consis = np.maximum(PR9_consis_pos, PR9_consis_neg)
    
    PR9 = np.power(PR9_CNRM*PR9_NorESM*PR9_CESM2*PR9_CESM2_gw*PR9_IPSL*PR9_E3SM, 1/6)
#     PR9 = np.power(PR9_NorESM*PR9_CESM2*PR9_CESM2_gw*PR9_IPSL*PR9_MIROC, 1/5)
    PR9[data_mask==0] = np.nan
    return PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR1_consis,PR2_consis,PR3_consis,PR4_consis,PR5_consis,PR6_consis,PR7_consis,PR8_consis,PR9_consis


def plot_PR_single(ax, data_xarray, data_signal, title, metric, perc):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('cool')
    Reds = mpl.cm.get_cmap('coolwarm')
    
    colors_Blues = [Blues(0.3), Blues(0.2), Blues(0.1), Blues(0.0), 'lightgray', Reds(0.7), Reds(0.8), Reds(0.9), Reds(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [1/4, 1/3, 1/2, 2/3, 1.5, 2, 3, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks([0.25, 0.33333333, 0.5, 0.66666667, 1.5, 2, 3, 4])
    cb.set_ticklabels(["1/4", "1/3", "1/2", "2/3", "3/2", "2", "3", "4"])
    cb.set_label(label = 'Probability Ratio', fontsize=14)
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + ' ' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
#     ax.contourf(data_lon, data_lat, data_signal, levels=[4.5, 5.5, 6.5], hatches=['ooo', '***'], colors= 'none')
    ax.contourf(data_lon, data_lat, data_signal, levels=[0, 3.5], colors= 'lightgray')
#     ax.contour(data_lon, data_lat, data_signal, levels=[3.5, 6.5], colors= 'k', linewidths=0.4)
    
    
def plot_PR_single_no_cb(ax, data_xarray, data_signal, title, metric, perc):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('cool')
    Reds = mpl.cm.get_cmap('coolwarm')
    
    colors_Blues = [Blues(0.3), Blues(0.2), Blues(0.1), Blues(0.0), 'lightgray', Reds(0.7), Reds(0.8), Reds(0.9), Reds(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [1/4, 1/3, 1/2, 2/3, 1.5, 2, 3, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
#     cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
#     cb.ax.tick_params(labelsize=12)
#     cb.set_ticks([0.25, 0.33333333, 0.5, 0.66666667, 1.5, 2, 3, 4])
#     cb.set_ticklabels(["1/4", "1/3", "1/2", "2/3", "3/2", "2", "3", "4"])
#     cb.set_label(label = 'Probability Ratio', fontsize=14)
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + ' ' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
#     ax.contourf(data_lon, data_lat, data_signal, levels=[4.5, 5.5, 6.5], hatches=['ooo', '***'], colors= 'none')
    ax.contourf(data_lon, data_lat, data_signal, levels=[0, 3.5], colors= 'lightgray')
#     ax.contour(data_lon, data_lat, data_signal, levels=[3.5, 6.5], colors= 'k', linewidths=0.4)

In [ ]:
def get_data_for_this_study_for_AT(metric):
    
    
    PR1_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_NorESM= np.array(PR1_NorESM)
    PR1_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_CESM2= np.array(PR1_CESM2)
    PR1_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_1')
    PR1_CESM2_gw= np.array(PR1_CESM2_gw)
    PR1_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
    PR1_IPSL= np.array(PR1_IPSL)
#     PR1_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
#     PR1_MIROC= np.array(PR1_MIROC)
    PR1_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_all')
    PR1_E3SM= np.array(PR1_E3SM)
    
    PR1_consis_pos = np.zeros([163, 288])
    PR1_consis_neg = np.zeros([163, 288])
    
    PR1_consis_pos[PR1_NorESM>=1.5] = PR1_consis_pos[PR1_NorESM>=1.5]+1
    PR1_consis_neg[PR1_NorESM<2/3] = PR1_consis_neg[PR1_NorESM<2/3]+1
    
    PR1_consis_pos[PR1_CESM2>=1.5] = PR1_consis_pos[PR1_CESM2>=1.5]+1
    PR1_consis_neg[PR1_CESM2<2/3] = PR1_consis_neg[PR1_CESM2<2/3]+1
    
    PR1_consis_pos[PR1_CESM2_gw>=1.5] = PR1_consis_pos[PR1_CESM2_gw>=1.5]+1
    PR1_consis_neg[PR1_CESM2_gw<2/3] = PR1_consis_neg[PR1_CESM2_gw<2/3]+1
    
    PR1_consis_pos[PR1_IPSL>=1.5] = PR1_consis_pos[PR1_IPSL>=1.5]+1
    PR1_consis_neg[PR1_IPSL<2/3] = PR1_consis_neg[PR1_IPSL<2/3]+1
    
#     PR1_consis_pos[PR1_MIROC>=1.5] = PR1_consis_pos[PR1_MIROC>=1.5]+1
#     PR1_consis_neg[PR1_MIROC<2/3] = PR1_consis_neg[PR1_MIROC<2/3]+1
    
    PR1_consis_pos[PR1_E3SM>=1.5] = PR1_consis_pos[PR1_E3SM>=1.5]+1
    PR1_consis_neg[PR1_E3SM<2/3] = PR1_consis_neg[PR1_E3SM<2/3]+1
    
    PR1_consis = np.maximum(PR1_consis_pos, PR1_consis_neg)
    
    scio.savemat("/dodrio/scratch/users/vsc10154/PR1_consis.mat", {'PR1_consis': PR1_consis})
    
    PR1 = np.power(PR1_NorESM*PR1_CESM2*PR1_CESM2_gw*PR1_IPSL*PR1_E3SM, 1/5)
#     PR1 = np.power(PR1_NorESM*PR1_CESM2*PR1_CESM2_gw*PR1_IPSL*PR1_MIROC, 1/5)
    PR1[data_mask==0] = np.nan
    
    
    
    
    
    PR2_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_NorESM= np.array(PR2_NorESM)
    PR2_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_CESM2= np.array(PR2_CESM2)
    PR2_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_2')
    PR2_CESM2_gw= np.array(PR2_CESM2_gw)
    PR2_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
    PR2_IPSL= np.array(PR2_IPSL)
#     PR2_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
#     PR2_MIROC= np.array(PR2_MIROC)
    PR2_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_oth')
    PR2_E3SM= np.array(PR2_E3SM)
    PR2 = np.power(PR2_NorESM*PR2_CESM2*PR2_CESM2_gw*PR2_IPSL*PR2_E3SM, 1/5)
#     PR2 = np.power(PR2_NorESM*PR2_CESM2*PR2_CESM2_gw*PR2_IPSL*PR2_MIROC, 1/5)
    
    PR2[data_mask==0] = np.nan
    
    PR2_consis_pos = np.zeros([163, 288])
    PR2_consis_neg = np.zeros([163, 288])
    
    PR2_consis_pos[PR2_NorESM>=1.5] = PR2_consis_pos[PR2_NorESM>=1.5]+1
    PR2_consis_neg[PR2_NorESM<2/3] = PR2_consis_neg[PR2_NorESM<2/3]+1
    
    PR2_consis_pos[PR2_CESM2>=1.5] = PR2_consis_pos[PR2_CESM2>=1.5]+1
    PR2_consis_neg[PR2_CESM2<2/3] = PR2_consis_neg[PR2_CESM2<2/3]+1
    
    PR2_consis_pos[PR2_CESM2_gw>=1.5] = PR2_consis_pos[PR2_CESM2_gw>=1.5]+1
    PR2_consis_neg[PR2_CESM2_gw<2/3] = PR2_consis_neg[PR2_CESM2_gw<2/3]+1
    
    PR2_consis_pos[PR2_IPSL>=1.5] = PR2_consis_pos[PR2_IPSL>=1.5]+1
    PR2_consis_neg[PR2_IPSL<2/3] = PR2_consis_neg[PR2_IPSL<2/3]+1
    
#     PR2_consis_pos[PR2_MIROC>=1.5] = PR2_consis_pos[PR2_MIROC>=1.5]+1
#     PR2_consis_neg[PR2_MIROC<2/3] = PR2_consis_neg[PR2_MIROC<2/3]+1
    
    PR2_consis_pos[PR2_E3SM>=1.5] = PR2_consis_pos[PR2_E3SM>=1.5]+1
    PR2_consis_neg[PR2_E3SM<2/3] = PR2_consis_neg[PR2_E3SM<2/3]+1
    
    PR2_consis = np.maximum(PR2_consis_pos, PR2_consis_neg)
    
    
    PR3_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_NorESM= np.array(PR3_NorESM)
    PR3_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_CESM2= np.array(PR3_CESM2)
    PR3_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%.mat', 'PR_3')
    PR3_CESM2_gw= np.array(PR3_CESM2_gw)
    PR3_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
    PR3_IPSL= np.array(PR3_IPSL)
#     PR3_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
#     PR3_MIROC= np.array(PR3_MIROC)
    PR3_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.0%_0.9x1.25.mat', 'PR_irr')
    PR3_E3SM= np.array(PR3_E3SM)
    PR3 = np.power(PR3_NorESM*PR3_CESM2*PR3_CESM2_gw*PR3_IPSL*PR3_E3SM, 1/5)
#     PR3 = np.power(PR3_NorESM*PR3_CESM2*PR3_CESM2_gw*PR3_IPSL*PR3_MIROC, 1/5)
    
    PR3[data_mask==0] = np.nan
    
    PR3_consis_pos = np.zeros([163, 288])
    PR3_consis_neg = np.zeros([163, 288])
    
    PR3_consis_pos[PR3_NorESM>=1.5] = PR3_consis_pos[PR3_NorESM>=1.5]+1
    PR3_consis_neg[PR3_NorESM<2/3] = PR3_consis_neg[PR3_NorESM<2/3]+1
    
    PR3_consis_pos[PR3_CESM2>=1.5] = PR3_consis_pos[PR3_CESM2>=1.5]+1
    PR3_consis_neg[PR3_CESM2<2/3] = PR3_consis_neg[PR3_CESM2<2/3]+1
    
    PR3_consis_pos[PR3_CESM2_gw>=1.5] = PR3_consis_pos[PR3_CESM2_gw>=1.5]+1
    PR3_consis_neg[PR3_CESM2_gw<2/3] = PR3_consis_neg[PR3_CESM2_gw<2/3]+1
    
    PR3_consis_pos[PR3_IPSL>=1.5] = PR3_consis_pos[PR3_IPSL>=1.5]+1
    PR3_consis_neg[PR3_IPSL<2/3] = PR3_consis_neg[PR3_IPSL<2/3]+1
    
#     PR3_consis_pos[PR3_MIROC>=1.5] = PR3_consis_pos[PR3_MIROC>=1.5]+1
#     PR3_consis_neg[PR3_MIROC<2/3] = PR3_consis_neg[PR3_MIROC<2/3]+1
    
    PR3_consis_pos[PR3_E3SM>=1.5] = PR3_consis_pos[PR3_E3SM>=1.5]+1
    PR3_consis_neg[PR3_E3SM<2/3] = PR3_consis_neg[PR3_E3SM<2/3]+1
    
    PR3_consis = np.maximum(PR3_consis_pos, PR3_consis_neg)
    
    
    PR4_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_NorESM= np.array(PR4_NorESM)
    PR4_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_CESM2= np.array(PR4_CESM2)
    PR4_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_4')
    PR4_CESM2_gw= np.array(PR4_CESM2_gw)
    PR4_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
    PR4_IPSL= np.array(PR4_IPSL)
#     PR4_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
#     PR4_MIROC= np.array(PR4_MIROC)
    PR4_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_all')
    PR4_E3SM= np.array(PR4_E3SM)
    PR4 = np.power(PR4_NorESM*PR4_CESM2*PR4_CESM2_gw*PR4_IPSL*PR4_E3SM, 1/5)
#     PR4 = np.power(PR4_NorESM*PR4_CESM2*PR4_CESM2_gw*PR4_IPSL*PR4_MIROC, 1/5)
    PR4[data_mask==0] = np.nan
    
    PR4_consis_pos = np.zeros([163, 288])
    PR4_consis_neg = np.zeros([163, 288])
    
    PR4_consis_pos[PR4_NorESM>=1.5] = PR4_consis_pos[PR4_NorESM>=1.5]+1
    PR4_consis_neg[PR4_NorESM<2/3] = PR4_consis_neg[PR4_NorESM<2/3]+1
    
    PR4_consis_pos[PR4_CESM2>=1.5] = PR4_consis_pos[PR4_CESM2>=1.5]+1
    PR4_consis_neg[PR4_CESM2<2/3] = PR4_consis_neg[PR4_CESM2<2/3]+1
    
    PR4_consis_pos[PR4_CESM2_gw>=1.5] = PR4_consis_pos[PR4_CESM2_gw>=1.5]+1
    PR4_consis_neg[PR4_CESM2_gw<2/3] = PR4_consis_neg[PR4_CESM2_gw<2/3]+1
    
    PR4_consis_pos[PR4_IPSL>=1.5] = PR4_consis_pos[PR4_IPSL>=1.5]+1
    PR4_consis_neg[PR4_IPSL<2/3] = PR4_consis_neg[PR4_IPSL<2/3]+1
    
#     PR4_consis_pos[PR4_MIROC>=1.5] = PR4_consis_pos[PR4_MIROC>=1.5]+1
#     PR4_consis_neg[PR4_MIROC<2/3] = PR4_consis_neg[PR4_MIROC<2/3]+1
    
    PR4_consis_pos[PR4_E3SM>=1.5] = PR4_consis_pos[PR4_E3SM>=1.5]+1
    PR4_consis_neg[PR4_E3SM<2/3] = PR4_consis_neg[PR4_E3SM<2/3]+1
    
    PR4_consis = np.maximum(PR4_consis_pos, PR4_consis_neg)
    
    
    
    PR5_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_NorESM= np.array(PR5_NorESM)
    PR5_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_CESM2= np.array(PR5_CESM2)
    PR5_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_5')
    PR5_CESM2_gw= np.array(PR5_CESM2_gw)
    PR5_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
    PR5_IPSL= np.array(PR5_IPSL)
#     PR5_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
#     PR5_MIROC= np.array(PR5_MIROC)
    PR5_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_oth')
    PR5_E3SM= np.array(PR5_E3SM)
    PR5 = np.power(PR5_NorESM*PR5_CESM2*PR5_CESM2_gw*PR5_IPSL*PR5_E3SM, 1/5)
#     PR5 = np.power(PR5_NorESM*PR5_CESM2*PR5_CESM2_gw*PR5_IPSL*PR5_MIROC, 1/5)
    PR5[data_mask==0] = np.nan
    
    
    PR5_consis_pos = np.zeros([163, 288])
    PR5_consis_neg = np.zeros([163, 288])
    
    PR5_consis_pos[PR5_NorESM>=1.5] = PR5_consis_pos[PR5_NorESM>=1.5]+1
    PR5_consis_neg[PR5_NorESM<2/3] = PR5_consis_neg[PR5_NorESM<2/3]+1
    
    PR5_consis_pos[PR5_CESM2>=1.5] = PR5_consis_pos[PR5_CESM2>=1.5]+1
    PR5_consis_neg[PR5_CESM2<2/3] = PR5_consis_neg[PR5_CESM2<2/3]+1
    
    PR5_consis_pos[PR5_CESM2_gw>=1.5] = PR5_consis_pos[PR5_CESM2_gw>=1.5]+1
    PR5_consis_neg[PR5_CESM2_gw<2/3] = PR5_consis_neg[PR5_CESM2_gw<2/3]+1
    
    PR5_consis_pos[PR5_IPSL>=1.5] = PR5_consis_pos[PR5_IPSL>=1.5]+1
    PR5_consis_neg[PR5_IPSL<2/3] = PR5_consis_neg[PR5_IPSL<2/3]+1
    
#     PR5_consis_pos[PR5_MIROC>=1.5] = PR5_consis_pos[PR5_MIROC>=1.5]+1
#     PR5_consis_neg[PR5_MIROC<2/3] = PR5_consis_neg[PR5_MIROC<2/3]+1
    
    PR5_consis_pos[PR5_E3SM>=1.5] = PR5_consis_pos[PR5_E3SM>=1.5]+1
    PR5_consis_neg[PR5_E3SM<2/3] = PR5_consis_neg[PR5_E3SM<2/3]+1
    
    PR5_consis = np.maximum(PR5_consis_pos, PR5_consis_neg)
    
    
    PR6_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_NorESM= np.array(PR6_NorESM)
    PR6_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_CESM2= np.array(PR6_CESM2)
    PR6_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%.mat', 'PR_6')
    PR6_CESM2_gw= np.array(PR6_CESM2_gw)
    PR6_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
    PR6_IPSL= np.array(PR6_IPSL)
#     PR6_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
#     PR6_MIROC= np.array(PR6_MIROC)
    PR6_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.5%_0.9x1.25.mat', 'PR_irr')
    PR6_E3SM= np.array(PR6_E3SM)
    PR6 = np.power(PR6_NorESM*PR6_CESM2*PR6_CESM2_gw*PR6_IPSL*PR6_E3SM, 1/5)
#     PR6 = np.power(PR6_NorESM*PR6_CESM2*PR6_CESM2_gw*PR6_IPSL*PR6_MIROC, 1/5)
    PR6[data_mask==0] = np.nan


    PR6_consis_pos = np.zeros([163, 288])
    PR6_consis_neg = np.zeros([163, 288])
    
    PR6_consis_pos[PR6_NorESM>=1.5] = PR6_consis_pos[PR6_NorESM>=1.5]+1
    PR6_consis_neg[PR6_NorESM<2/3] = PR6_consis_neg[PR6_NorESM<2/3]+1
    
    PR6_consis_pos[PR6_CESM2>=1.5] = PR6_consis_pos[PR6_CESM2>=1.5]+1
    PR6_consis_neg[PR6_CESM2<2/3] = PR6_consis_neg[PR6_CESM2<2/3]+1
    
    PR6_consis_pos[PR6_CESM2_gw>=1.5] = PR6_consis_pos[PR6_CESM2_gw>=1.5]+1
    PR6_consis_neg[PR6_CESM2_gw<2/3] = PR6_consis_neg[PR6_CESM2_gw<2/3]+1
    
    PR6_consis_pos[PR6_IPSL>=1.5] = PR6_consis_pos[PR6_IPSL>=1.5]+1
    PR6_consis_neg[PR6_IPSL<2/3] = PR6_consis_neg[PR6_IPSL<2/3]+1
    
#     PR6_consis_pos[PR6_MIROC>=1.5] = PR6_consis_pos[PR6_MIROC>=1.5]+1
#     PR6_consis_neg[PR6_MIROC<2/3] = PR6_consis_neg[PR6_MIROC<2/3]+1
    
    PR6_consis_pos[PR6_E3SM>=1.5] = PR6_consis_pos[PR6_E3SM>=1.5]+1
    PR6_consis_neg[PR6_E3SM<2/3] = PR6_consis_neg[PR6_E3SM<2/3]+1
    
    PR6_consis = np.maximum(PR6_consis_pos, PR6_consis_neg)

    
    
    PR7_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_NorESM= np.array(PR7_NorESM)
    PR7_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_CESM2= np.array(PR7_CESM2)
    PR7_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_all_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_7')
    PR7_CESM2_gw= np.array(PR7_CESM2_gw)
    PR7_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_all_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
    PR7_IPSL= np.array(PR7_IPSL)
#     PR7_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_all_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
#     PR7_MIROC= np.array(PR7_MIROC)
    PR7_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_all_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_all')
    PR7_E3SM= np.array(PR7_E3SM)
    PR7 = np.power(PR7_NorESM*PR7_CESM2*PR7_CESM2_gw*PR7_IPSL*PR7_E3SM, 1/5)
#     PR7 = np.power(PR7_NorESM*PR7_CESM2*PR7_CESM2_gw*PR7_IPSL*PR7_MIROC, 1/5)
    PR7[data_mask==0] = np.nan
    
    
    PR7_consis_pos = np.zeros([163, 288])
    PR7_consis_neg = np.zeros([163, 288])
    
    PR7_consis_pos[PR7_NorESM>=1.5] = PR7_consis_pos[PR7_NorESM>=1.5]+1
    PR7_consis_neg[PR7_NorESM<2/3] = PR7_consis_neg[PR7_NorESM<2/3]+1
    
    PR7_consis_pos[PR7_CESM2>=1.5] = PR7_consis_pos[PR7_CESM2>=1.5]+1
    PR7_consis_neg[PR7_CESM2<2/3] = PR7_consis_neg[PR7_CESM2<2/3]+1
    
    PR7_consis_pos[PR7_CESM2_gw>=1.5] = PR7_consis_pos[PR7_CESM2_gw>=1.5]+1
    PR7_consis_neg[PR7_CESM2_gw<2/3] = PR7_consis_neg[PR7_CESM2_gw<2/3]+1
    
    PR7_consis_pos[PR7_IPSL>=1.5] = PR7_consis_pos[PR7_IPSL>=1.5]+1
    PR7_consis_neg[PR7_IPSL<2/3] = PR7_consis_neg[PR7_IPSL<2/3]+1
    
#     PR7_consis_pos[PR7_MIROC>=1.5] = PR7_consis_pos[PR7_MIROC>=1.5]+1
#     PR7_consis_neg[PR7_MIROC<2/3] = PR7_consis_neg[PR7_MIROC<2/3]+1
    
    PR7_consis_pos[PR7_E3SM>=1.5] = PR7_consis_pos[PR7_E3SM>=1.5]+1
    PR7_consis_neg[PR7_E3SM<2/3] = PR7_consis_neg[PR7_E3SM<2/3]+1
    
    PR7_consis = np.maximum(PR7_consis_pos, PR7_consis_neg)
    
    
    PR8_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_NorESM= np.array(PR8_NorESM)
    PR8_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_CESM2= np.array(PR8_CESM2)
    PR8_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_8')
    PR8_CESM2_gw= np.array(PR8_CESM2_gw)
    PR8_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_oth_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
    PR8_IPSL= np.array(PR8_IPSL)
#     PR8_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_oth_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
#     PR8_MIROC= np.array(PR8_MIROC)
    PR8_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_oth_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_oth')
    PR8_E3SM= np.array(PR8_E3SM)
    PR8 = np.power(PR8_NorESM*PR8_CESM2*PR8_CESM2_gw*PR8_IPSL*PR8_E3SM, 1/5)
#     PR8 = np.power(PR8_NorESM*PR8_CESM2*PR8_CESM2_gw*PR8_IPSL*PR8_MIROC, 1/5)
    PR8[data_mask==0] = np.nan
    
    
    PR8_consis_pos = np.zeros([163, 288])
    PR8_consis_neg = np.zeros([163, 288])
    
    PR8_consis_pos[PR8_NorESM>=1.5] = PR8_consis_pos[PR8_NorESM>=1.5]+1
    PR8_consis_neg[PR8_NorESM<2/3] = PR8_consis_neg[PR8_NorESM<2/3]+1
    
    PR8_consis_pos[PR8_CESM2>=1.5] = PR8_consis_pos[PR8_CESM2>=1.5]+1
    PR8_consis_neg[PR8_CESM2<2/3] = PR8_consis_neg[PR8_CESM2<2/3]+1
    
    PR8_consis_pos[PR8_CESM2_gw>=1.5] = PR8_consis_pos[PR8_CESM2_gw>=1.5]+1
    PR8_consis_neg[PR8_CESM2_gw<2/3] = PR8_consis_neg[PR8_CESM2_gw<2/3]+1
    
    PR8_consis_pos[PR8_IPSL>=1.5] = PR8_consis_pos[PR8_IPSL>=1.5]+1
    PR8_consis_neg[PR8_IPSL<2/3] = PR8_consis_neg[PR8_IPSL<2/3]+1
    
#     PR8_consis_pos[PR8_MIROC>=1.5] = PR8_consis_pos[PR8_MIROC>=1.5]+1
#     PR8_consis_neg[PR8_MIROC<2/3] = PR8_consis_neg[PR8_MIROC<2/3]+1
    
    PR8_consis_pos[PR8_E3SM>=1.5] = PR8_consis_pos[PR8_E3SM>=1.5]+1
    PR8_consis_neg[PR8_E3SM<2/3] = PR8_consis_neg[PR8_E3SM<2/3]+1
    
    PR8_consis = np.maximum(PR8_consis_pos, PR8_consis_neg)
    
    
    PR9_NorESM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_NorESM= np.array(PR9_NorESM)
    PR9_CESM2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_CESM2= np.array(PR9_CESM2)
    PR9_CESM2_gw = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%.mat', 'PR_9')
    PR9_CESM2_gw= np.array(PR9_CESM2_gw)
    PR9_IPSL = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/IPSL-CM6/PR/PR_irr_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
    PR9_IPSL= np.array(PR9_IPSL)
#     PR9_MIROC = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/PR_irr_' +\
#                                              metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
#     PR9_MIROC= np.array(PR9_MIROC)
    PR9_E3SM = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/E3SM_cdo/PR_irr_' +\
                                             metric + '_'+ '99.9%_0.9x1.25.mat', 'PR_irr')
    PR9_E3SM= np.array(PR9_E3SM)
    
    PR9_consis_pos = np.zeros([163, 288])
    PR9_consis_neg = np.zeros([163, 288])
     
    PR9_consis_pos[PR9_NorESM>=1.5] = PR9_consis_pos[PR9_NorESM>=1.5]+1
    PR9_consis_neg[PR9_NorESM<2/3] = PR9_consis_neg[PR9_NorESM<2/3]+1
    
    PR9_consis_pos[PR9_CESM2>=1.5] = PR9_consis_pos[PR9_CESM2>=1.5]+1
    PR9_consis_neg[PR9_CESM2<2/3] = PR9_consis_neg[PR9_CESM2<2/3]+1
    
    PR9_consis_pos[PR9_CESM2_gw>=1.5] = PR9_consis_pos[PR9_CESM2_gw>=1.5]+1
    PR9_consis_neg[PR9_CESM2_gw<2/3] = PR9_consis_neg[PR9_CESM2_gw<2/3]+1
    
    PR9_consis_pos[PR9_IPSL>=1.5] = PR9_consis_pos[PR9_IPSL>=1.5]+1
    PR9_consis_neg[PR9_IPSL<2/3] = PR9_consis_neg[PR9_IPSL<2/3]+1
    
#     PR9_consis_pos[PR9_MIROC>=1.5] = PR9_consis_pos[PR9_MIROC>=1.5]+1
#     PR9_consis_neg[PR9_MIROC<2/3] = PR9_consis_neg[PR9_MIROC<2/3]+1
    
    PR9_consis_pos[PR9_E3SM>=1.5] = PR9_consis_pos[PR9_E3SM>=1.5]+1
    PR9_consis_neg[PR9_E3SM<2/3] = PR9_consis_neg[PR9_E3SM<2/3]+1
    
    PR9_consis = np.maximum(PR9_consis_pos, PR9_consis_neg)
    
    PR9 = np.power(PR9_NorESM*PR9_CESM2*PR9_CESM2_gw*PR9_IPSL*PR9_E3SM, 1/5)
#     PR9 = np.power(PR9_NorESM*PR9_CESM2*PR9_CESM2_gw*PR9_IPSL*PR9_MIROC, 1/5)
    PR9[data_mask==0] = np.nan
    return PR1,PR2,PR3,PR4,PR5,PR6,PR7,PR8,PR9,PR1_consis,PR2_consis,PR3_consis,PR4_consis,PR5_consis,PR6_consis,PR7_consis,PR8_consis,PR9_consis


def plot_PR_single_for_AT(ax, data_xarray, data_signal, title, metric, perc):
    cmap = 'coolwarm'
    Blues = mpl.cm.get_cmap('cool')
    Reds = mpl.cm.get_cmap('coolwarm')
    
    colors_Blues = [Blues(0.3), Blues(0.2), Blues(0.1), Blues(0.0), 'lightgray', Reds(0.7), Reds(0.8), Reds(0.9), Reds(1.0)]
    cmap = LinearSegmentedColormap.from_list('cmap_name', colors_Blues, N=9)
    vmin=1/4
    vmax=4
    alpha=0.05
    levels = [1/4, 1/3, 1/2, 2/3, 1.5, 2, 3, 4]
    im = data_xarray.plot(ax=ax, cmap=cmap,vmin=vmin,vmax=vmax,levels=levels, extend='both', add_colorbar=False, add_labels=False)
    cb = plt.colorbar(im,fraction= 0.3, pad= 0.04, extend='both', orientation='horizontal')
    cb.ax.tick_params(labelsize=12)
    cb.set_ticks([0.25, 0.33333333, 0.5, 0.66666667, 1.5, 2, 3, 4])
    cb.set_ticklabels(["1/4", "1/3", "1/2", "2/3", "3/2", "2", "3", "4"])
    ax.coastlines(color='dimgray', linewidth=0.5)
    ax.add_feature(cfeature.OCEAN, color='lightgrey')
    ax.set_title(title, loc='right', fontsize=14)
    ax.set_title(metric + '_' + perc, loc='left', fontsize=14)
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.1)
    ax.contourf(data_lon, data_lat, data_signal, levels=[0, 3.5], colors= 'lightgray')
#     ax.contour(data_lon, data_lat, data_signal, levels=[3.5, 7], colors= 'k', linewidths=0.4)


In [ ]:
def all_for_paper():
    metric = 'TSA'
    varname = '$\mathregular{T_{2m}}$'
    PR_1,PR_2,PR_3,PR_4,PR_5,PR_6,PR_7,PR_8,PR_9,PR1_consis,PR2_consis,PR3_consis,PR4_consis,PR5_consis,PR6_consis,PR7_consis,PR8_consis,PR9_consis = get_data_for_this_study(metric)


    fig = plt.figure(figsize=(18,12),dpi=300)
    fig.subplots_adjust(hspace=0.05, wspace=0.05, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
    proj=ccrs.PlateCarree()


    data_xr_7 = xr.DataArray(PR_7, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(331, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_7, PR7_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'a', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')


    data_xr_8 = xr.DataArray(PR_8, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(334, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_8, PR8_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'd', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_9 = xr.DataArray(PR_9, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(337, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, PR9_consis, 'IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'g', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    
    metric = 'HUMIDEX'
    varname = '$\mathregular{HU}$'
    PR_1,PR_2,PR_3,PR_4,PR_5,PR_6,PR_7,PR_8,PR_9,PR1_consis,PR2_consis,PR3_consis,PR4_consis,PR5_consis,PR6_consis,PR7_consis,PR8_consis,PR9_consis = get_data_for_this_study(metric)



    data_xr_7 = xr.DataArray(PR_7, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(332, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_7, PR7_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'b', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')


    data_xr_8 = xr.DataArray(PR_8, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(335, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_8, PR8_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'e', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_9 = xr.DataArray(PR_9, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(338, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, PR9_consis, 'IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'h', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')
    
    metric = 'T_w'
    varname = '$\mathregular{T_{w}}$'
    PR_1,PR_2,PR_3,PR_4,PR_5,PR_6,PR_7,PR_8,PR_9,PR1_consis,PR2_consis,PR3_consis,PR4_consis,PR5_consis,PR6_consis,PR7_consis,PR8_consis,PR9_consis = get_data_for_this_study(metric)



    data_xr_7 = xr.DataArray(PR_7, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(333, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_7, PR7_consis, 'ALL', varname, '99.9%')
    ax.text(0.01, 0.92, 'c', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')


    data_xr_8 = xr.DataArray(PR_8, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(336, projection=proj, frameon=False)
    plot_PR_single_no_cb(ax, data_xr_8, PR8_consis, 'ALL-IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'f', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

    data_xr_9 = xr.DataArray(PR_9, coords={'y': data_lat, 'x': data_lon}, dims=["y", "x"])
    ax = plt.subplot(339, projection=proj, frameon=False)
    plot_PR_single(ax, data_xr_9, PR9_consis, 'IE', varname, '99.9%')
    ax.text(0.01, 0.92, 'i', color='dimgrey', fontsize=10, transform=ax.transAxes, weight='bold')

In [ ]:
all_for_paper()